##### Copyright 2018 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

**Multi-Label Image Classification With Tensorflow And Keras
Note:**

Multi-label classification is a type of classification in which an object can be categorized into more than one class. For example, In the above dataset, we will classify a picture as the image of a dog or cat and also classify the same image based on the breed of the dog or cat.
Multi-class classification is simply classifying objects into any one of multiple categories. Such as classifying just into either a dog or cat from the dataset above.

## Import TensorFlow and other libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls '/content'

drive  sample_data


In [5]:
!nvidia-smi

Fri Oct  1 07:53:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

2.6.0


In [7]:
!unzip -q '/content/drive/My Drive/Pets/unseen.zip'

In [8]:
!ls '/content/'

drive  sample_data  unseen


In [9]:
!ls '/content/unseen/unseen_train'

img_0001.jpg  img_1455.jpg  img_2900.jpg  img_4361.jpg	img_5821.jpg
img_0002.jpg  img_1456.jpg  img_2901.jpg  img_4364.jpg	img_5824.jpg
img_0003.jpg  img_1457.jpg  img_2902.jpg  img_4365.jpg	img_5825.jpg
img_0004.jpg  img_1458.jpg  img_2904.jpg  img_4367.jpg	img_5827.jpg
img_0005.jpg  img_1459.jpg  img_2905.jpg  img_4369.jpg	img_5828.jpg
img_0006.jpg  img_1460.jpg  img_2906.jpg  img_4370.jpg	img_5829.jpg
img_0007.jpg  img_1461.jpg  img_2907.jpg  img_4371.jpg	img_5830.jpg
img_0008.jpg  img_1462.jpg  img_2908.jpg  img_4372.jpg	img_5831.jpg
img_0009.jpg  img_1465.jpg  img_2909.jpg  img_4373.jpg	img_5832.jpg
img_0010.jpg  img_1466.jpg  img_2910.jpg  img_4374.jpg	img_5833.jpg
img_0011.jpg  img_1467.jpg  img_2911.jpg  img_4376.jpg	img_5834.jpg
img_0012.jpg  img_1468.jpg  img_2912.jpg  img_4377.jpg	img_5835.jpg
img_0014.jpg  img_1469.jpg  img_2913.jpg  img_4378.jpg	img_5836.jpg
img_0015.jpg  img_1470.jpg  img_2914.jpg  img_4379.jpg	img_5837.jpg
img_0016.jpg  img_1471.jpg  img_2915.jpg  img_43

**Preparing the training data**

To feed the images into the Neural Network we would require the images to be loaded. We are provided with a train.csv file consisting of the image names and the respective categories. We can use the sheet to load the images using the flow_from_dataframe method from Keras, but the method requires the complete filename with the extension of the image. Since we have jpg images we will format all the id’s in train.csv by adding ‘.jpg’ to all rows. We will then create a new training set with 3 columns namely Images, Animal and Breed. Since the Animal and Breed columns are categories we will convert the type to string.

In [10]:
#Prepare training set for breed
training_set_b = pd.read_csv('/content/unseen/train_unseen.csv')

training_imgs = ["{}.jpg".format(x) for x in list(training_set_b.id)]

training_labels_2 = list(training_set_b['breed'])
training_set_b = pd.DataFrame( {'Images': training_imgs,'breed' : training_labels_2})
training_set_b
#Changing the type  to str
training_set_b.breed = training_set_b.breed.astype(str)

In [11]:
#Prepare training set for species

training_set = pd.read_csv('/content/unseen/train_unseen.csv')

training_imgs = ["{}.jpg".format(x) for x in list(training_set.id)]

training_labels_1 = list(training_set['class_name'])
training_set = pd.DataFrame( {'Images': training_imgs,'Animal': training_labels_1})

#Changing the type  to str
training_set.Animal = training_set.Animal.astype(str)

In [12]:
print(training_set.head())

         Images Animal
0  img_0001.jpg      2
1  img_0002.jpg      1
2  img_0003.jpg      2
3  img_0004.jpg      2
4  img_0005.jpg      2


In [13]:
print(training_set_b.head())

         Images breed
0  img_0001.jpg    24
1  img_0002.jpg    15
2  img_0003.jpg    24
3  img_0004.jpg    21
4  img_0005.jpg    22


**Preprocessing Images**

We will now preprocess the images using Keras’ ImageDataGenerator class which will convert the images into an array of vectors that can be fed to the neural network. A set of features or parameters can be initialized to the ImageDataGenerator such as rescale, shear_range, zoom_range etc. These parameters help in extracting maximum features from an image.

The flow_from_dataframe method allows us to import images from a data frame provided the path of the images using the parameter ‘directory’. The x_col specifies the independent factor which is an image and y_col represents the dependent factor which is the category of the image that we need to predict. The target size will be the size of the resulting images from the ImageDataGenerator object, batch_size is the number of sample images used to train at once.

In [14]:
#Processing for Species
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  validation_split = 0.1)

train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory='/content/unseen/unseen_train',x_col="Images",
                                        y_col="Animal",
                                        class_mode="sparse",
                                        target_size=(128,128),
                                        batch_size=32)

validation_generator = train_dataGen.flow_from_dataframe(
    dataframe = training_set,
    directory='/content/unseen/unseen_train',x_col="Images",
    y_col="Animal",
    class_mode="sparse",
    target_size=(128,128),
    batch_size=32,
    subset='validation') # set as validation data

Found 6123 validated image filenames belonging to 2 classes.
Found 612 validated image filenames belonging to 2 classes.


In [15]:
#Processing for Breeds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  validation_split = 0.1)

train_generator_b = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set_b,
                                        directory='/content/unseen/unseen_train',x_col="Images",
                                        y_col="breed",
                                        class_mode="sparse",
                                        target_size=(128,128),
                                        batch_size=32)

validation_generator_b = train_dataGen.flow_from_dataframe(
    dataframe = training_set_b,
    directory='/content/unseen/unseen_train',x_col="Images",
    y_col="breed",
    class_mode="sparse",
    target_size=(128,128),
    batch_size=32,
    subset='validation') # set as validation data

Found 6123 validated image filenames belonging to 10 classes.
Found 612 validated image filenames belonging to 10 classes.


**Building Convolutional Neural Network**

We are done processing the image data. Now we can proceed to build a simple convolutional neural network. Keras allows us to build neural networks effortlessly with a couple of classes and methods.

The Sequential class initializes a network to which we can add layers and nodes.

The add method allows us to add layers of nodes to the initialized network. In the above code, we added a Convolutional layer to the network. The convolution will be performed using a 3×3 matrix as specified with the kernel _size parameter. The activation parameter sets the activation function for the nodes. The input size should be same as the size of the outputs from the ImageDataGenerator (3 is the channel width).

The above code adds a pooling layer to the network.

We can add as many layers as we want as shown below, however, this puts a lot of pressure on the system resources. Choose the layers and nodes based on the capability of the machines.

The line below adds a Flattening layer to the network.

Next, we will add a hidden layer and an output layer to complete the network as done with the following code blocks.

We will now compile the network to initialize the metrics, loss and weights for the network

Let’s have a look at the description of our CNN :


**Model definition and complie common for Species/Breed**

In [16]:
classifier = Sequential()
#First Convolutional layer
classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)))
classifier.add(MaxPooling2D(pool_size = (2,2)))
#second Convolutional layer
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
#Flattening
classifier.add(Flatten())
#Hidden Layer
classifier.add(Dense(units = 64, activation = 'relu'))

#Output Layer
classifier.add(Dense(units = 10 , activation = 'softmax'))

In [17]:
classifier.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [18]:
# # Model Fitting for Breed
# classifier.fit_generator(
#     train_generator_b,
#     steps_per_epoch = train_generator_b.samples // 32,
#     validation_data = validation_generator_b, 
#     validation_steps = validation_generator_b.samples // 32,
#     epochs = 30)

In [19]:
# Model Fitting for Species
classifier.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 32,
    epochs = 10)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
191/191 [==============================] - 70s 200ms/step - loss: 0.6365 - accuracy: 0.6799 - val_loss: 0.5143 - val_accuracy: 0.7418
Epoch 2/10
191/191 [==============================] - 36s 187ms/step - loss: 0.5062 - accuracy: 0.7610 - val_loss: 0.4293 - val_accuracy: 0.8191
Epoch 3/10
191/191 [==============================] - 36s 188ms/step - loss: 0.4753 - accuracy: 0.7805 - val_loss: 0.4756 - val_accuracy: 0.7796
Epoch 4/10
191/191 [==============================] - 36s 188ms/step - loss: 0.4528 - accuracy: 0.7954 - val_loss: 0.4163 - val_accuracy: 0.8306
Epoch 5/10
191/191 [==============================] - 36s 187ms/step - loss: 0.4349 - accuracy: 0.8002 - val_loss: 0.4080 - val_accuracy: 0.8240
Epoch 6/10
191/191 [==============================] - 36s 187ms/step - loss: 0.4315 - accuracy: 0.8099 - val_loss: 0.3766 - val_accuracy: 0.8438
Epoch 7/10
191/191 [==============================] - 36s 188ms/step - loss: 0.4148 - accuracy: 0.8179 - val_loss: 0.3675 - val_ac

In [20]:
classifier.save('/content/drive/My Drive/Models/UnseenPetImageClassificationSpecies.h5')

In [21]:
# classifier_s.save('/content/drive/My Drive/Models/PetImageClassificationSpecies.h5')

**Test Set Preparation common for Species/Breed**

We will prepare the test data by adding the path and file extension to the original test_set. This will help us load the images directly from the csv file using the load_img() method that you will see in the following code blocks.

In [22]:
!unzip -q '/content/drive/My Drive/Pets/unseen_test.zip'

In [23]:
!ls '/content/unseen_test/'

img_7301.jpg  img_7323.jpg  img_7345.jpg  img_7363.jpg	img_7384.jpg
img_7303.jpg  img_7324.jpg  img_7346.jpg  img_7364.jpg	img_7385.jpg
img_7305.jpg  img_7325.jpg  img_7347.jpg  img_7365.jpg	img_7386.jpg
img_7306.jpg  img_7327.jpg  img_7348.jpg  img_7366.jpg	img_7387.jpg
img_7307.jpg  img_7328.jpg  img_7349.jpg  img_7367.jpg	img_7388.jpg
img_7308.jpg  img_7329.jpg  img_7350.jpg  img_7368.jpg	img_7389.jpg
img_7309.jpg  img_7330.jpg  img_7351.jpg  img_7370.jpg	img_7391.jpg
img_7311.jpg  img_7331.jpg  img_7352.jpg  img_7372.jpg	img_7392.jpg
img_7312.jpg  img_7333.jpg  img_7353.jpg  img_7373.jpg	img_7393.jpg
img_7314.jpg  img_7334.jpg  img_7354.jpg  img_7375.jpg	img_7394.jpg
img_7315.jpg  img_7336.jpg  img_7355.jpg  img_7376.jpg	img_7396.jpg
img_7316.jpg  img_7337.jpg  img_7356.jpg  img_7377.jpg	img_7397.jpg
img_7317.jpg  img_7339.jpg  img_7357.jpg  img_7378.jpg	img_7398.jpg
img_7318.jpg  img_7340.jpg  img_7358.jpg  img_7380.jpg	img_7399.jpg
img_7319.jpg  img_7341.jpg  img_7359.jpg  img_73

In [24]:
import os
files = next(os.walk('/content/unseen_test'))[2]
dirs = next(os.walk('/content/unseen_test/UnseenSpecies'))[1]
dirb = next(os.walk('/content/unseen_test/UnseenBreed'))[1]


In [25]:
class_names = dirs
# del class_names[0]
class_names = sorted(class_names)
class_names

['11', '12', '13', '14', '15', '21', '22', '23', '24', '25']

In [26]:
dic = {k: v for v, k in enumerate(class_names)}
dic
thisdict = dict(map(reversed, dic.items()))
thisdict

{0: '11',
 1: '12',
 2: '13',
 3: '14',
 4: '15',
 5: '21',
 6: '22',
 7: '23',
 8: '24',
 9: '25'}

In [27]:
data = []
for file in files:
  data.append((file))

df = pd.DataFrame(data, columns=['id'])
df = df.sort_values(by='id', ascending=True)
df

,id
52,img_7301.jpg
22,img_7303.jpg
41,img_7305.jpg
23,img_7306.jpg
50,img_7307.jpg
...,...
9,img_7396.jpg
61,img_7397.jpg
11,img_7398.jpg
49,img_7399.jpg


In [28]:
import pandas as pd
test_set = df
test_imgs = ['/content/unseen_test/{}'.format(x) for x in list(test_set.id)]
test_set = pd.DataFrame( {'Images': test_imgs })
test_set = test_set.sort_values(by='Images', ascending=True)

In [29]:
# test_set = pd.read_csv('/content/test.csv')
# test_imgs = ['/content/images_test/{}.jpg'.format(x) for x in list(test_set.id)]
# test_set = pd.DataFrame( {'Images': test_imgs })

In [30]:
test_set

,Images
0,/content/unseen_test/img_7301.jpg
1,/content/unseen_test/img_7303.jpg
2,/content/unseen_test/img_7305.jpg
3,/content/unseen_test/img_7306.jpg
4,/content/unseen_test/img_7307.jpg
...,...
78,/content/unseen_test/img_7396.jpg
79,/content/unseen_test/img_7397.jpg
80,/content/unseen_test/img_7398.jpg
81,/content/unseen_test/img_7399.jpg


**Inference for Breed**

**Identifying the trained classes**

The train_generator consists of the complete trained image data. Let’s have a look at the unique categories in the training data using the class_indices attribute of the train_generator. 

In [31]:
classes = train_generator_b.class_indices
print(classes)

{'11': 0, '12': 1, '13': 2, '14': 3, '15': 4, '21': 5, '22': 6, '23': 7, '24': 8, '25': 9}


Our model will be predicting the labels in the range 0 to 9 based on the above dictionary for each category. We will need to reverse these to the original classes. We will use a reverse of the above dictionary to later convert the predictions to actual classes. The dictionary can be inverted with the following line of code:

In [32]:
reversed_dictionary = {value : key for (key, value) in classes.items()}
print(reversed_dictionary)

{0: '11', 1: '12', 2: '13', 3: '14', 4: '15', 5: '21', 6: '22', 7: '23', 8: '24', 9: '25'}


In [33]:
#Load breed model
classifier_b = tf.keras.models.load_model('/content/drive/My Drive/Models/PetImageClassificationBreed.h5')

**Predicting classes**

Now it’s time to load the images one by one and predict and store the category of each image from the test_set. 

The above code block loads each image from the test set preprocess it and feeds it to the classifier to predict. The predictions are stored in a list called y_pred. Notice that all the image sizes (128,128) are the same as we did for the training set, this is important and otherwise would result in an error.

In [34]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(128,128,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,128,128,3))
  img_class = classifier_b.predict(test_img)
  prediction = img_class[0]
  Y_pred.append(prediction)

In [35]:
# print(Y_pred)

In [36]:
ypred = tf.argmax(Y_pred, axis=1)

In [37]:
pred = tf.Variable(ypred).numpy().tolist()

In [38]:
print(pred)

[6, 1, 9, 6, 9, 7, 6, 2, 6, 6, 6, 9, 1, 6, 6, 5, 6, 9, 1, 1, 6, 8, 7, 1, 1, 1, 7, 6, 6, 6, 7, 9, 9, 9, 1, 7, 7, 6, 7, 8, 2, 7, 1, 1, 1, 7, 6, 5, 6, 6, 7, 7, 7, 9, 6, 1, 6, 7, 6, 6, 7, 6, 7, 9, 1, 9, 7, 1, 9, 7, 6, 7, 7, 1, 7, 7, 7, 9, 1, 6, 9, 9, 7]


In [39]:
prediction_classes = [reversed_dictionary.get(item,item) for item in pred]

In [40]:
print(prediction_classes)

['22', '12', '25', '22', '25', '23', '22', '13', '22', '22', '22', '25', '12', '22', '22', '21', '22', '25', '12', '12', '22', '24', '23', '12', '12', '12', '23', '22', '22', '22', '23', '25', '25', '25', '12', '23', '23', '22', '23', '24', '13', '23', '12', '12', '12', '23', '22', '21', '22', '22', '23', '23', '23', '25', '22', '12', '22', '23', '22', '22', '23', '22', '23', '25', '12', '25', '23', '12', '25', '23', '22', '23', '23', '12', '23', '23', '23', '25', '12', '22', '25', '25', '23']


In [41]:
#animal = []
breed = []
for i in prediction_classes:
 # animal.append(i[0]) # First character = class_name/Animal
  breed.append(i[0:]) # Last 2 characters = breed/Breed 

In [42]:
predictions = {}
#predictions['class_name'] = animal
predictions['breed'] = breed

In [43]:
prediction = pd.DataFrame(predictions)

In [44]:
prediction

,breed
0,22
1,12
2,25
3,22
4,25
...,...
78,12
79,22
80,25
81,25


In [45]:
df

,id
52,img_7301.jpg
22,img_7303.jpg
41,img_7305.jpg
23,img_7306.jpg
50,img_7307.jpg
...,...
9,img_7396.jpg
61,img_7397.jpg
11,img_7398.jpg
49,img_7399.jpg


In [46]:
prediction['breed'].value_counts()

22    24
23    23
12    16
25    14
13     2
21     2
24     2
Name: breed, dtype: int64

In [47]:
detail_result = df.join(prediction)
detail_result 

,id,breed
52,img_7301.jpg,23
22,img_7303.jpg,23
41,img_7305.jpg,23
23,img_7306.jpg,12
50,img_7307.jpg,23
...,...,...
9,img_7396.jpg,22
61,img_7397.jpg,22
11,img_7398.jpg,25
49,img_7399.jpg,22


In [48]:
# prediction.groupby('breed').count()

**Inference for Species**

In [49]:
classes = train_generator.class_indices
print(classes)

{'1': 0, '2': 1}


In [50]:
reversed_dictionary = {value : key for (key, value) in classes.items()}
print(reversed_dictionary)

{0: '1', 1: '2'}


In [51]:
#Load Species Model
classifier_s = tf.keras.models.load_model('/content/drive/My Drive/Models/PetImageClassificationSpecies.h5')

In [52]:
Y_pred = []

for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(128,128,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,128,128,3))
  img_class = classifier_s.predict(test_img)
  prediction = img_class[0]
  Y_pred.append(prediction)

In [53]:
# print(Y_pred)

In [54]:
ypred = tf.argmax(Y_pred, axis=1)

pred = tf.Variable(ypred).numpy().tolist()

print(pred)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]


In [55]:
prediction_classes = [reversed_dictionary.get(item,item) for item in pred]

print(prediction_classes)

['2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '1', '2', '2', '2', '2', '2', '2', '1', '2', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '1', '2', '2', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2']


In [56]:
animal = []
for i in prediction_classes:
  animal.append(i[0]) # First character = class_name/Animal
  
predictions = {}
predictions['animal'] = animal

prediction = pd.DataFrame(predictions)

In [57]:
prediction['animal'].value_counts()

2    74
1     9
Name: animal, dtype: int64

In [58]:
detail_result = df.join(prediction)
detail_result 

,id,animal
52,img_7301.jpg,2
22,img_7303.jpg,2
41,img_7305.jpg,2
23,img_7306.jpg,2
50,img_7307.jpg,2
...,...,...
9,img_7396.jpg,2
61,img_7397.jpg,2
11,img_7398.jpg,2
49,img_7399.jpg,2


In [59]:
# #Writing to excel
pd.DataFrame(detail_result).to_excel('/content/drive/My Drive/Pets/detail_result.xlsx', index = False)